In [1]:
import os
import fnmatch

import numpy as np
import matplotlib.pyplot as plt

from astropy.io import fits
from astropy.table import Table, vstack, hstack

## KiDS multi-band catalog

In [2]:
local_kids_dir = '/data2/lsajkov/mpdg/data/KiDS_multiband'

### RA/Dec catalog:

In [ ]:
KiDS_positions_catalog = Table()

for kids_multiband_cat in fnmatch.filter(os.listdir(local_kids_dir),
                                         'KiDS_DR4.1*ugriZYJHKs_cat.fits'):
    
    with fits.open(f'{local_kids_dir}/{kids_multiband_cat}') as cat:
        catData = Table(cat[1].data)
    
    KiDS_positions_catalog = vstack([KiDS_positions_catalog,
                                     catData['ID', 'RAJ2000', 'DECJ2000']])

In [ ]:
KiDS_positions_catalog.write('/data2/lsajkov/mpdg/data_products/KiDS/KiDS_concat_radec_positions.fits',
                             overwrite = True)

### Photometry catalog:

ID, position, size, 9-band photometry

In [ ]:
#desired data columns
with fits.open(f'{local_kids_dir}/{fnmatch.filter(os.listdir(local_kids_dir), 'KiDS_DR4.1*ugriZYJHKs_cat.fits')[0]}') as hdu:
    hduData = Table(hdu[1].data)
    
ID_cols = ['ID', 'KIDS_TILE', 'THELI_NAME']

position_cols = ['RAJ2000', 'DECJ2000']

size_cols = ['FWHM_WORLD']

MAG_GAAP_bands    = [band for band in hduData.keys()\
                  if (fnmatch.fnmatch(band, 'MAG_GAAP_*') & (not fnmatch.fnmatch(band, 'MAG_GAAP_*_*')))]

MAGERR_GAAP_bands = [band for band in hduData.keys()\
                  if (fnmatch.fnmatch(band, 'MAGERR_GAAP_*') & (not fnmatch.fnmatch(band, 'MAGERR_GAAP_*_*')))]

COLOUR_GAAP_bands = [band for band in hduData.keys()\
                  if fnmatch.fnmatch(band, 'COLOUR_GAAP_*')]

index_array =  [ID_cols,
                position_cols,
                size_cols,
                MAG_GAAP_bands,
                MAGERR_GAAP_bands,
                COLOUR_GAAP_bands]

column_indices = [index for array in index_array for index in array]

In [ ]:
KiDS_concatenated_catalog = Table()

for kids_multiband_cat in fnmatch.filter(os.listdir(local_kids_dir),
                                         'KiDS_DR4.1*ugriZYJHKs_cat.fits'):
    
    with fits.open(f'{local_kids_dir}/{kids_multiband_cat}') as cat:
        catData = Table(cat[1].data)
    
    KiDS_concatenated_catalog = vstack([KiDS_concatenated_catalog,
                                     catData[column_indices]])

In [ ]:
def 

### SOM catalog
Only within primary targeting region <br>
Color-color catalog

In [30]:
masking_info = {'0':	'THELI manual mask (very conservative)',
                '1':	'THELI automatic large star halo mask (faint)',
                '2':	'THELI automatic large star halo mask (bright) or bright star mask',
                '3':	'Manual mask of regions around globular clusters, Fornax dwarf, ISS passage',
                '4':	'THELI void mask, or asteroids, or weight=0',
                '5':	'VIKING Z-band image masked',
                '6':	'VIKING Y-band image masked',
                '7':	'VIKING J-band image masked',
                '8':	'VIKING H-band image masked',
                '9':	'VIKING Ks-band image masked',
                '10':	'Astro-WISE u-band halo+stellar PULECENELLA mask or weight=0',
                '11':	'Astro-WISE g-band halo+stellar PULECENELLA mask or weight=0',
                '12':	'Astro-WISE r-band halo+stellar PULECENELLA mask or weight=0',
                '13':	'Astro-WISE i-band halo+stellar PULECENELLA mask or weight=0',
                '14':	'Object outside the RA/DEC cut for this tile',
                '15':	'Not used (reserved as sign bit in FITS 2-byte integer)}'}

priyanka_mask = 28668
mijin_mask    = 27676

priyanka_mask_bin = f'{priyanka_mask:b}'
mijin_mask_bin    = f'{mijin_mask:b}'

print('Bit\tPriy.\tMijin\tMeaning')
print('--------------------------------')
for i in range(15):
    print(f'{i}\t{bool(int(priyanka_mask_bin[-(i + 1)]))}\t{bool(int(mijin_mask_bin[-(i + 1)]))}\t{masking_info[f"{i}"]}')

Bit	Priy.	Mijin	Meaning
--------------------------------
0	False	False	THELI manual mask (very conservative)
1	False	False	THELI automatic large star halo mask (faint)
2	True	True	THELI automatic large star halo mask (bright) or bright star mask
3	True	True	Manual mask of regions around globular clusters, Fornax dwarf, ISS passage
4	True	True	THELI void mask, or asteroids, or weight=0
5	True	False	VIKING Z-band image masked
6	True	False	VIKING Y-band image masked
7	True	False	VIKING J-band image masked
8	True	False	VIKING H-band image masked
9	True	False	VIKING Ks-band image masked
10	True	True	Astro-WISE u-band halo+stellar PULECENELLA mask or weight=0
11	True	True	Astro-WISE g-band halo+stellar PULECENELLA mask or weight=0
12	False	False	Astro-WISE r-band halo+stellar PULECENELLA mask or weight=0
13	True	True	Astro-WISE i-band halo+stellar PULECENELLA mask or weight=0
14	True	True	Object outside the RA/DEC cut for this tile


In [3]:
#define primary targeting region
def gr_color_condition(gr_color, gr_color_err,
               r_mag):
    
    return (gr_color - gr_color_err + 0.06 * (r_mag - 14)) < 0.9

In [50]:
KiDS_SOM_catalog = Table()

for kids_multiband_cat in fnmatch.filter(os.listdir(local_kids_dir),
                                         'KiDS_DR4.*_*ugriZYJHKs_cat.fits'):
    
    with fits.open(f'{local_kids_dir}/{kids_multiband_cat}') as cat:
        catData = Table(cat[1].data)

    #define columns
    ID = catData['ID']

    ra = catData['RAJ2000']
    dec = catData['DECJ2000']

    u_SNR = catData['FLUX_GAAP_u']/catData['FLUXERR_GAAP_u']
    g_SNR = catData['FLUX_GAAP_g']/catData['FLUXERR_GAAP_g']
    r_SNR = catData['FLUX_GAAP_r']/catData['FLUXERR_GAAP_r']
    i_SNR = catData['FLUX_GAAP_i']/catData['FLUXERR_GAAP_i']
    Z_SNR = catData['FLUX_GAAP_Z']/catData['FLUXERR_GAAP_Z']

    r_mag = catData['MAG_AUTO']

    # u_mag     = catData['MAG_GAAP_u']
    # g_mag     = catData['MAG_GAAP_g']
    # r_mag     = catData['MAG_GAAP_r']
    # i_mag     = catData['MAG_GAAP_i']
    # Z_mag     = catData['MAG_GAAP_Z']

    g_mag_err = catData['MAGERR_GAAP_g']
    u_mag_err = catData['MAGERR_GAAP_u']
    r_mag_err = catData['MAGERR_GAAP_r']
    i_mag_err = catData['MAGERR_GAAP_i']
    Z_mag_err = catData['MAGERR_GAAP_Z']

    ug_color     = catData['COLOUR_GAAP_u_g']
    ug_color_err = np.sqrt(u_mag_err ** 2 + g_mag_err ** 2)    

    gr_color     = catData['COLOUR_GAAP_g_r']
    gr_color_err = np.sqrt(r_mag_err ** 2 + g_mag_err ** 2)

    ri_color     = catData['COLOUR_GAAP_r_i']
    ri_color_err = np.sqrt(r_mag_err ** 2 + i_mag_err ** 2)

    iZ_color     = catData['COLOUR_GAAP_i_Z']
    iZ_color_err = np.sqrt(i_mag_err ** 2 + Z_mag_err ** 2)

    redshift = catData['Z_B']
    redshift_err = (catData['Z_B_MAX'] - catData['Z_B_MIN'])/2

    #define masks
    KiDS_flags_mask = catData['FLAG_GAAP_r'] == 0

    KiDS_MASK_mask = ~((catData['MASK'] & mijin_mask) > 0)

    redshift_mask = redshift < 1

    flux_radius = catData['FLUX_RADIUS'] * 0.213

    gr_color_mask = gr_color_condition(gr_color,
                                       gr_color_err,
                                       r_mag)
    
    color_mask = (ug_color > -1) & (ug_color < 2) &\
                 (gr_color > -1) & (gr_color < 2) &\
                 (ri_color > -1) & (ri_color < 2) &\
                 (iZ_color > -1) & (iZ_color < 2)
    
    SNR_thresh = 10

    SNR_mask = (u_SNR > SNR_thresh) &\
               (g_SNR > SNR_thresh) &\
               (r_SNR > SNR_thresh) &\
               (i_SNR > SNR_thresh) &\
               (Z_SNR > SNR_thresh)

    r_mag_mask = r_mag > 18

    complete_mask = KiDS_flags_mask &\
                    KiDS_MASK_mask &\
                    gr_color_mask &\
                    color_mask &\
                    r_mag_mask &\
                    SNR_mask &\
                    redshift_mask #&\
                    #surf_bright_mask

    tile_KiDS_data = hstack([ID,
                             ra, dec,
                            #  u_mag, u_mag_err,
                            #  g_mag, g_mag_err,
                             r_mag, r_mag_err,
                            #  i_mag, i_mag_err,
                            #  Z_mag, Z_mag_err,
                             ug_color, ug_color_err,
                             gr_color, gr_color_err,
                             ri_color, ri_color_err,
                             iZ_color, iZ_color_err,
                             flux_radius,
                             redshift, redshift_err
                             ]) 
    tile_KiDS_data.rename_columns(tile_KiDS_data.colnames,
                           ['ID',
                            'ra', 'dec',
                            # 'u_mag', 'u_mag_err',
                            # 'g_mag', 'g_mag_err',
                            'r_mag', 'r_mag_err',
                            # 'i_mag', 'i_mag_err',
                            # 'Z_mag', 'Z_mag_err',
                            'ug_col', 'ug_col_err',
                            'gr_col', 'gr_col_err',
                            'ri_col', 'ri_col_err',
                            'iZ_col', 'iZ_col_err',
                            'half_light_radius',
                            'redshift', 'redshift_err'
                            ])
    
    tile_KiDS_data = tile_KiDS_data[complete_mask]

    KiDS_SOM_catalog = vstack([KiDS_SOM_catalog,
                               tile_KiDS_data])

In [14]:
with fits.open(local_kids_dir + '/' + os.listdir(local_kids_dir)[0]) as cat:
    test_table = Table(cat[1].data)
    test_header = cat[1].header

In [15]:
test_header

XTENSION= 'BINTABLE'           / binary table extension                         
BITPIX  =                    8 / array data type                                
NAXIS   =                    2 / number of array dimensions                     
NAXIS1  =                 1270 / length of dimension 1                          
NAXIS2  =                87556 / length of dimension 2                          
PCOUNT  =                    0 / number of group parameters                     
GCOUNT  =                    1 / number of groups                               
TFIELDS =                  306 / number of table fields                         
TTYPE1  = 'ID      '           / ESO_ID IAU astronomical source designation     
TINDX1  = 'T       '                                                            
TUCD1   = 'meta.id;meta.main'                                                   
TUNIT1  = ''                                                                    
TCOMM1  = 'ESO_ID IAU astron

In [8]:
test_table['MAG_AUTO'] + test_table['DMAG'] - test_table['EXTINCTION_r']

KeyError: 'DMAG'

In [10]:
test_table.keys()

['ID',
 'KIDS_TILE',
 'THELI_NAME',
 'SeqNr',
 'SLID',
 'SID',
 'FLUX_AUTO',
 'FLUXERR_AUTO',
 'MAG_AUTO',
 'MAGERR_AUTO',
 'KRON_RADIUS',
 'BackGr',
 'Level',
 'MU_THRESHOLD',
 'MaxVal',
 'MU_MAX',
 'ISOAREA_WORLD',
 'Xpos',
 'Ypos',
 'RAJ2000',
 'DECJ2000',
 'A_WORLD',
 'B_WORLD',
 'THETA_J2000',
 'THETA_WORLD',
 'ERRA_WORLD',
 'ERRB_WORLD',
 'ERRTHETA_J2000',
 'ERRTHETA_WORLD',
 'FWHM_IMAGE',
 'FWHM_WORLD',
 'Flag',
 'FLUX_RADIUS',
 'CLASS_STAR',
 'EXTINCTION_u',
 'EXTINCTION_g',
 'EXTINCTION_r',
 'EXTINCTION_i',
 'Agaper_0p7',
 'Bgaper_0p7',
 'Agaper_1p0',
 'Bgaper_1p0',
 'PAgaap',
 'FLAG_GAAP_0p7_u',
 'FLAG_GAAP_0p7_g',
 'FLAG_GAAP_0p7_r',
 'FLAG_GAAP_0p7_i',
 'FLUX_GAAP_0p7_u',
 'FLUX_GAAP_0p7_g',
 'FLUX_GAAP_0p7_r',
 'FLUX_GAAP_0p7_i',
 'FLUXERR_GAAP_0p7_u',
 'FLUXERR_GAAP_0p7_g',
 'FLUXERR_GAAP_0p7_r',
 'FLUXERR_GAAP_0p7_i',
 'FLAG_GAAP_1p0_u',
 'FLAG_GAAP_1p0_g',
 'FLAG_GAAP_1p0_r',
 'FLAG_GAAP_1p0_i',
 'FLUX_GAAP_1p0_u',
 'FLUX_GAAP_1p0_g',
 'FLUX_GAAP_1p0_r',
 'FLUX_GAAP_1p0

In [17]:
#Using Bilicki+21 selection criteria

KiDS_SOM_catalog = Table()

for kids_multiband_cat in fnmatch.filter(os.listdir(local_kids_dir),
                                         'KiDS_DR4.*_*ugriZYJHKs_cat.fits'):
    
    with fits.open(f'{local_kids_dir}/{kids_multiband_cat}') as cat:
        catData = Table(cat[1].data)

    #define columns
    ID = catData['ID']

    ra = catData['RAJ2000']
    dec = catData['DECJ2000']

    u_SNR = catData['FLUX_GAAP_u']/catData['FLUXERR_GAAP_u']
    g_SNR = catData['FLUX_GAAP_g']/catData['FLUXERR_GAAP_g']
    r_SNR = catData['FLUX_GAAP_r']/catData['FLUXERR_GAAP_r']
    i_SNR = catData['FLUX_GAAP_i']/catData['FLUXERR_GAAP_i']
    Z_SNR = catData['FLUX_GAAP_Z']/catData['FLUXERR_GAAP_Z']

    r_mag = catData['MAG_AUTO']

    # u_mag     = catData['MAG_GAAP_u']
    # g_mag     = catData['MAG_GAAP_g']
    # r_mag     = catData['MAG_GAAP_r']
    # i_mag     = catData['MAG_GAAP_i']
    # Z_mag     = catData['MAG_GAAP_Z']

    g_mag_err = catData['MAGERR_GAAP_g']
    u_mag_err = catData['MAGERR_GAAP_u']
    r_mag_err = catData['MAGERR_GAAP_r']
    i_mag_err = catData['MAGERR_GAAP_i']
    Z_mag_err = catData['MAGERR_GAAP_Z']

    ug_color     = catData['COLOUR_GAAP_u_g']
    ug_color_err = np.sqrt(u_mag_err ** 2 + g_mag_err ** 2)    

    gr_color     = catData['COLOUR_GAAP_g_r']
    gr_color_err = np.sqrt(r_mag_err ** 2 + g_mag_err ** 2)

    ri_color     = catData['COLOUR_GAAP_r_i']
    ri_color_err = np.sqrt(r_mag_err ** 2 + i_mag_err ** 2)

    iZ_color     = catData['COLOUR_GAAP_i_Z']
    iZ_color_err = np.sqrt(i_mag_err ** 2 + Z_mag_err ** 2)

    redshift = catData['Z_B']
    redshift_err = (catData['Z_B_MAX'] - catData['Z_B_MIN'])/2

    #define masks
    # KiDS_flags_mask = catData['FLAG_GAAP_r'] == 0

    KiDS_MASK_mask         = ~((catData['MASK'] & 28668) > 0)
    KiDS_IMAFLAGS_ISO_mask = (catData['IMAFLAGS_ISO'] == 0)
    KiDS_CLASS_STAR_mask   = catData['CLASS_STAR'] < 0.5
    KiDS_SG2DPHOT_mask     = catData['SG2DPHOT'] == 0
    KiDS_SGFLAG_mask       = catData['SG_FLAG'] == 1

    redshift_mask = redshift < 1

    flux_radius = catData['FLUX_RADIUS'] * 0.213

    gr_color_mask = gr_color_condition(gr_color,
                                       gr_color_err,
                                       r_mag)
    
    color_mask = (ug_color > -1) & (ug_color < 2) &\
                 (gr_color > -1) & (gr_color < 2) &\
                 (ri_color > -1) & (ri_color < 2) &\
                 (iZ_color > -1) & (iZ_color < 2)
    
    SNR_thresh = 10

    SNR_mask = (u_SNR > SNR_thresh) &\
               (g_SNR > SNR_thresh) &\
               (r_SNR > SNR_thresh) &\
               (i_SNR > SNR_thresh) &\
               (Z_SNR > SNR_thresh)

    r_mag_mask = r_mag > 18

    complete_mask = KiDS_MASK_mask &\
                    KiDS_IMAFLAGS_ISO_mask &\
                    KiDS_CLASS_STAR_mask &\
                    KiDS_SG2DPHOT_mask &\
                    KiDS_SGFLAG_mask &\
                    gr_color_mask &\
                    color_mask &\
                    r_mag_mask &\
                    SNR_mask &\
                    redshift_mask #&\
                    #surf_bright_mask

    tile_KiDS_data = hstack([ID,
                             ra, dec,
                            #  u_mag, u_mag_err,
                            #  g_mag, g_mag_err,
                             r_mag, r_mag_err,
                            #  i_mag, i_mag_err,
                            #  Z_mag, Z_mag_err,
                             ug_color, ug_color_err,
                             gr_color, gr_color_err,
                             ri_color, ri_color_err,
                             iZ_color, iZ_color_err,
                             flux_radius,
                             redshift, redshift_err
                             ]) 
    tile_KiDS_data.rename_columns(tile_KiDS_data.colnames,
                           ['ID',
                            'ra', 'dec',
                            # 'u_mag', 'u_mag_err',
                            # 'g_mag', 'g_mag_err',
                            'r_mag', 'r_mag_err',
                            # 'i_mag', 'i_mag_err',
                            # 'Z_mag', 'Z_mag_err',
                            'ug_col', 'ug_col_err',
                            'gr_col', 'gr_col_err',
                            'ri_col', 'ri_col_err',
                            'iZ_col', 'iZ_col_err',
                            'half_light_radius',
                            'redshift', 'redshift_err'
                            ])
    
    tile_KiDS_data = tile_KiDS_data[complete_mask]

    KiDS_SOM_catalog = vstack([KiDS_SOM_catalog,
                               tile_KiDS_data])

In [52]:
KiDS_SOM_catalog.write('/data2/lsajkov/mpdg/data_products/KiDS/KiDS_SOM_catalog_18Jul24.fits',
                       overwrite = False)

### Log of catalogs
**Jul 12, 2024** <br>
*Included BPZ redshift in catalog* <br>
Cuts: <br>
$\mathrm{SNR} > 10$ <br>
$r > 18$ <br>
$-1 < color < 2$ <br>
In primary targeting region from Thornton+23 <br>

**Jul 15, 2024** <br>
*Included BPZ redshift in catalog* <br>
*Included half-light radius in catalog* <br>
Cuts: <br>
$\mathrm{SNR} > 10$ <br>
$r > 18$ <br>
$-1 < color < 2$ <br>
In primary targeting region from Thornton+23 <br>

**Jul 17, 2024** <br>
*Included BPZ redshift in catalog* <br>
*Included half-light radius in catalog* <br>
*Switched to using* MAG_AUTO *instead of* MAG_GAaP *for r-band. Removed other bands.* <br>
Cuts: <br>
$\mathrm{SNR} > 10$ <br>
$r > 18$ <br>
$-1 < color < 2$ <br>
In primary targeting region from Thornton+23 <br>

**Jul 18, 2024** <br>
*Included BPZ redshift in catalog* <br>
*Included half-light radius in catalog* <br>
*Switched to using* MAG_AUTO *instead of* MAG_GAaP *for r-band. Removed other bands.* <br>
*Included MASK cut.* <br>
*Inluded RA/Dec data.* <br>
*Expanded to use DR4.0+DR4.1 data* <br>
Cuts: <br>
$\mathrm{SNR} > 10$ <br>
$r > 18$ <br>
$-1 < color < 2$ <br>
$(\texttt{MASK}\ \&\ 27676) > 0$ (per Mijin) <br>
In primary targeting region from Thornton+23 <br>